In [1]:
import numpy as np,pandas as pd,json,sqlite3,requests, os
from jsonpath_ng import jsonpath
from jsonpath_ng.ext import parse
from pandas.io.json import json_normalize
from dotenv import load_dotenv
load_dotenv()
db_path = os.environ.get('dsa_db')
char_path = os.environ.get('char_data')
game_path = os.environ.get('game_data')
with open(char_path, 'r') as char_data:
    chars_json = json.load(char_data)

In [21]:
sherrif = chars_json['7266520144288097']

In [63]:
sherrif_df = pd.json_normalize(sherrif,record_path=['GearTierMeritItems'],meta=['ID'])
print(sherrif_df.columns)

Index(['TierGrantedAt', 'Icon', 'NameKey', 'UnitStats.Offense',
       'UnitStats.Defense', 'UnitStats.Health', 'UnitStats.Recovery',
       'UnitStats.CritChance', 'UnitStats.CritPower', 'UnitStats.AttackSpeed',
       'UnitStats.Speed', 'UnitStats.Potency', 'UnitStats.Tenacity',
       'UnitStats.Accuracy', 'UnitStats.Evasion', 'ID'],
      dtype='object')


In [61]:
sherrif_df = sherrif_df.loc[:,['ID','TierGrantedAt', 'UnitStats.Offense','UnitStats.Defense', 'UnitStats.Health']]

In [67]:
with sqlite3.connect(db_path) as conn:
    conn.execute('DROP TABLE IF EXISTS gear_merits')
    for toon in chars_json:

        toon_df = pd.json_normalize(chars_json[toon],record_path=['GearTierMeritItems'],meta=['ID'])
        try:
            toon_df = toon_df.loc[:,['ID','TierGrantedAt', 'UnitStats.Offense','UnitStats.Defense', 'UnitStats.Health']]
            toon_df.to_sql('gear_merits',conn, if_exists='append',index=False)
        except:
            continue
        


In [68]:
conn.commit()